# Sigma Protocols

A Sigma Protocol is a general three-move proof of knowledge. The goal is to prove the knowledge of some secret information, without revealing any information to the verifier, other than that we know the secret.

Alice wants to prove knowledge of a secret key to Bob. Of course, Alice doesn't want to reveal the secret key, just that she knows it. Here we'll use Schnorr's Proof of Identify, a well known exampel of a sigma protocol.

First, Alice has a secret key, $x$, and a public key, $H$:
 
$$ 
\begin{align*}
x &= \textrm{random element from} \, \{1, 2, 3, ..., q - 1\} \\
H &= g^x \mod q
\end{align*}
$$

where $q$ is a prime number and $g$ is a generator of a cyclic group with prime-order $q$. The public key, the group, and the generator are all shared publicly. Alice keeps the secret key and can use it to encrypt data.

This is the same key generation scheme as Diffie-Hellman and many other cryptographic schemes, which makes this protocol very useful. You could for instance use this protocol to prove knowledge of a private key used in an D-H exchange. 

As noted elsewhere, this takes advantage of the Discrete Logarithm problem. We are sharing the public key $H = g^x$, the generator $g$ is known as well. This means that if $\log_g(h)$ is easily calculated, anyone could find the private key $x$. By using a large prime $q$, which the generator $g$ is derived from, it's very difficult to compute $x$ from $h$.

First I'm going to set up Alice's private and public keys.

In [5]:
import numpy as np
from crypto import GeneratePrimeGeneratorPair

In [8]:
def generate_keys(q, g):
    secret = np.random.randint(1, q)
    public = pow(g, secret, q)
    
    return secret, public

In [27]:
q, g = GeneratePrimeGeneratorPair(15)
print(f"Prime: {q}\nGenerator: {g}")

sec_key, pub_key = generate_keys(q, g)
print(f"Secret Key: {sec_key}\nPublic Key: {pub_key}")

Prime: 23021
Generator: 12535
Secret Key: 12721
Public Key: 1685


Alice does this part, then publishes the prime, generator, and public key.

Now Alice will prove knowledge of the secret key to Bob using Schnorr's proof of identification, an example of a Sigma protocol.

The overall protocol looks like this:

![Schnorr's Protocol](Schnorrs-protocol.png)

First, Alice selects a random integer $k$ from $\{1,...,q\}$, then sends $h=g^k \bmod q$ to Bob. Bob responds with a challenge $c$, also a random integer drawn from $\{1,...,q\}$. 

Alice calculates $s$ and sends it to Bob.
$$\large s = k - cx \bmod q $$

Bob then verifies 
$$\large h = H^c g^s$$

This works because

$$
\large H^c g^s = (g^x)^c g^{k - cx} \bmod q = g^k \bmod q = h
$$

For this statement to be true, the shared key requires knowledge of the private key. 

As an exercise, try implementing this protocol using the secret and public keys created above.

In [34]:
# This doesn't work yet >_<

# Public information
public = (pub_key, q, g)

# The verifier creates a challenge and shares it with the prover
c = np.random.randint(1, q)
print(c)

# Solution
def prove(secret, challenge, q, g):
    # Returns h and s as defined above
    k = np.random.randint(1, q)
    h = pow(g, k, q)
    s = (k - secret * challenge) % q
    
    return h, s

def verify(public, challenge, h, s):
    # Returns True or False
    H, q, g = public
    print((pow(H, challenge, q) * pow(g, s, q)) % q)
    
    return h == (pow(H, challenge, q) * pow(g, s, q)) % q 
    
    
h, s = prove(sec_key, c, q, g)

print(h, s)

verify(public, c, h, s)

5812
4926 17792
9353


False

(Once I get this to work...)

Now something even more fun! Schnorr's protocol requires the prover to interact with the verifier, which means both parties need to be available at the same time and willing. It turns out we can make Schnorr's protocol non-interactive by choosing the challenge $c$ with a hash function.

...